In [3]:
import numpy as np
import pandas as pd
import os
import glob
import pickle

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn import neighbors, datasets
from sklearn.tree import DecisionTreeClassifier
path = 'LESSON07(13_11_2019)'

df2 = pd.DataFrame()
csv_file_path1 = glob.glob(os.path.join(path, "HW_13_wine_dataset", "*.csv"))
for file in csv_file_path1:
   
    df2 = df2.append(pd.read_csv(file, sep=';').assign(type=(file.split('-')[-1].split('.')[0])))

#df2["type"]= np.where(df2["type"] == 'red', 0, 1)
#'red' = 0, white = 1
df2 = df2.sample(frac=1) #перемішали

df2

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
2953,6.0,0.280,0.25,1.8,0.042,8.0,108.0,0.99290,3.08,0.55,9.0,5,white
1393,8.0,0.520,0.25,2.0,0.078,19.0,59.0,0.99612,3.30,0.48,10.2,5,red
1334,7.2,0.835,0.00,2.0,0.166,4.0,11.0,0.99608,3.39,0.52,10.0,5,red
591,7.3,0.200,0.37,1.2,0.037,48.0,119.0,0.99200,3.32,0.49,10.9,6,white
1508,6.8,0.210,0.49,14.5,0.060,50.0,170.0,0.99910,3.55,0.44,9.8,6,white
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1869,8.2,0.180,0.28,8.5,0.035,41.0,140.0,0.99520,3.04,0.37,10.1,7,white
1065,8.3,0.220,0.38,14.8,0.054,32.0,126.0,1.00020,3.22,0.50,9.7,5,white
3878,6.5,0.350,0.31,10.2,0.069,58.0,170.0,0.99692,3.18,0.49,9.4,5,white
4643,6.5,0.210,0.35,5.7,0.043,47.0,197.0,0.99392,3.24,0.50,10.1,6,white


In [4]:
df2['type'] = df2['type'].map({'red': 1, 'white': 0})

In [5]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6497 entries, 2953 to 332
Data columns (total 13 columns):
fixed acidity           6497 non-null float64
volatile acidity        6497 non-null float64
citric acid             6497 non-null float64
residual sugar          6497 non-null float64
chlorides               6497 non-null float64
free sulfur dioxide     6497 non-null float64
total sulfur dioxide    6497 non-null float64
density                 6497 non-null float64
pH                      6497 non-null float64
sulphates               6497 non-null float64
alcohol                 6497 non-null float64
quality                 6497 non-null int64
type                    6497 non-null int64
dtypes: float64(11), int64(2)
memory usage: 710.6 KB


In [6]:
y = df2['type'].astype('int')

In [7]:
X = df2.drop('type', axis=1)

In [8]:
X.shape, y.shape

((6497, 12), (6497,))

In [9]:
from sklearn.model_selection import train_test_split, cross_val_score
import numpy as np

In [10]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size=0.3, 
                                                      random_state=17)

In [11]:
X_train.shape, X_valid.shape

((4547, 12), (1950, 12))

In [12]:
first_tree = DecisionTreeClassifier(random_state=17)
#DecisionTreeClassifier - Класифікатор дерева рішень

In [13]:
np.mean(cross_val_score(first_tree, X_train, y_train, cv=5))

0.9837227239207437

In [14]:
from sklearn.neighbors import KNeighborsClassifier

In [15]:
first_knn = KNeighborsClassifier()

In [16]:
np.mean(cross_val_score(first_knn, X_train, y_train, cv=5)) #оцінка оцінки за допомогою перехресної перевірки 5- кількість складок у (Stratified)KFold)

0.9375391385292376

In [17]:
#настраиваем max_depth для дерева
from sklearn.model_selection import GridSearchCV


In [18]:
tree_params = {'max_depth': np.arange(1, 11), 'max_features':[.5, .7, 1]}

In [19]:
tree_grid = GridSearchCV(first_tree, tree_params, cv=5, n_jobs=-1)

In [20]:
%%time
tree_grid.fit(X_train, y_train);

Wall time: 5.82 s


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=17,
                                              splitter='best'),
             iid='warn', n_jobs=-1,
             param_grid={'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
                         'max_features': [0.5, 0.7, 1

In [21]:
tree_grid.best_score_, tree_grid.best_params_

(0.9865845612491753, {'max_depth': 9, 'max_features': 0.7})

In [22]:
knn_params = {'n_neighbors': range(5, 30, 5)} #+ list(range(50, 100, 10))}

In [23]:
knn_grid = GridSearchCV(first_knn, knn_params, cv=5)

In [24]:
%%time
knn_grid.fit(X_train, y_train);

Wall time: 3.04 s


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=None,
             param_grid={'n_neighbors': range(5, 30, 5)},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [25]:
knn_grid.best_score_, knn_grid.best_params_

(0.9401803386848472, {'n_neighbors': 10})

In [26]:
tree_valid_pred = tree_grid.predict(X_valid)

In [27]:
from sklearn.metrics import accuracy_score

In [28]:
accuracy_score(y_valid, tree_valid_pred)

0.9835897435897436

In [29]:
from sklearn.metrics import confusion_matrix

In [30]:
confusion_matrix(y_valid, tree_valid_pred)

array([[1469,   14],
       [  18,  449]], dtype=int64)

In [31]:
np.bincount(y_valid)

array([1483,  467], dtype=int64)

In [32]:
from sklearn.tree import export_graphviz

In [33]:
second_tree = DecisionTreeClassifier(max_depth=3).fit(X_train, y_train)
second_tree.score(X_valid, y_valid)

0.9682051282051282

In [34]:
export_graphviz(second_tree, out_file='telecom_tree2.dot',
               feature_names=X.columns, filled=True,)

In [35]:
!ls -l *.png

"ls" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [36]:
!dot -Tpng telecom_tree2.dot -o telecom_tree2.png

"dot" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.
